## PS1 Part 2: Unsupervised Linear Models

### Toy Dataset
For this problem, you will use the data file hb.csv. The input is 2,280 data points, each of which is 7
dimensional (i.e., input csv is 2280 rows by 7 columns). Use Principal Component Analysis
(either an existing library, or through your own implementation by taking the SVD of the
Covariance Matrix) for the follow tasks.

In [ ]:
%matplotlib inline
import pandas
url = "https://raw.githubusercontent.com/IDEALLab/ML4ME_Textbook/main/problems/hb.csv"
data = pandas.read_csv(url,header=None)
#data.head()

### Task 1
Assuming that the 7-dimensional space is excessive, you would like to reduce the dimension of the space. However, what dimensionality of space should we reduce it to? To determine this we need to compute its intrinsic dimensionality. Plot the relative value of the information content of each of the principal components and compare them.

Note: this information content is called the “explained variance” of each component, but you can also get this from the magnitude of the singular values. This plot is sometimes called a “Scree Plot”.

In [ ]:
# Code Here
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(data)

plt.figure(figsize=(8,8))
plt.pie(pca.explained_variance_ratio_,labels=[f"{pca.explained_variance_ratio_[i]*100:.4f}%" for i in range(7)])
plt.title("Principal Component relative values")
plt.show()

**Question:** Approximately how many components dominate the space?, and what does this tell us about the intrinsic dimensionality of the space?

**Response**: 2 components describe the space completely

#### Task 2
Now use PCA to project the 7-dimensional points on the K-dimensional space (where K is your answer from above) and plot the points. (For K=1,2, or 3, use a 1, 2, or 3D plot, respectively. For 4+ dimensions, use a grid of pairwise 2D Plots, like the Scatter Matrix we used in class).

In [ ]:
# Code Here
pca = PCA(n_components=2)
data_pca = pca.fit_transform(data)
print(data_pca)

plt.figure()
plt.scatter(data_pca[:,0],data_pca[:,1],marker='|')
plt.show()


**Question:** What do you notice?

**Response**: seems like the underlying dimension is formed like a skunk ^^

### Topology Optimization Dataset
For this problem, you will be using unsupervised linear models to help understand and interpret the results of a mechanical optimization problem. Specifically, to understand the solution space generated by a topology optimization code; that is, the results of finding the optimal geometries for minimizing the compliance of various bridge structures with different loading conditions. The input consists of 1,000 images of optimized material distribution for a beam as described in *Figure 1*. A symmetrical boundary condition, left side, is used to reduce the analysis to only half. Also, a rolling support is included at the lower right corner. Notice that the rolling support is the only support in the vertical direction.

&nbsp;

<center>
<img src="beam_description.jpg" alt="Beam description" style="max-width:100%;height:auto;">
</center>
<center>Figure 1: Left: Nx-by-Ny design domain for topology optimization problem. Right: Example loading configuration and resulting optimal topology. Two external forces, Fi, were applied to the beam at random nodes represented by (xi, yi) coordinates.<sup>1</sup></center>

&nbsp;

Use Principal Component Analysis (either an existing library, or through your own implementation by taking the SVD of the Covariance Matrix) for the follow tasks.

<sup>1. This problems data is based on the problem setup seen in the following paper: Ulu, E., Zhang, R., & Kara, L. B. (2016). A data-driven investigation and estimation of optimal topologies under variable loading configurations. *Computer Methods in Biomechanics and Biomedical Engineering: Imaging & Visualization*, 4(2), 61-72.</sup>

In [ ]:
# To help you get started, the below code will load the images from the associated image folder:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import requests
import zipfile
import io

#im_dir = './topo_opt_runs/'
url = "https://raw.githubusercontent.com/IDEALLab/ML4ME_Textbook/main/problems/topo_opt_runs.zip"
resp = requests.get(url)
resp.raise_for_status()
zf = zipfile.ZipFile(io.BytesIO(resp.content))

images = []
for name in sorted(zf.namelist()):
    with zf.open(name) as f:
        img = Image.open(f).convert('L')
        images.append(np.asarray(img))

height,width = images[0].shape
print('The images are {:d} pixels high and {:d} pixels wide'.format(height,width))

# Print matrix corresponding to the image:
print(images[-1])
# And show example image, so you can see how matrix correponds:

images = np.array(images)
images = images.reshape(1000,-1)
#check this again

print(images.shape)

img

### Task 1: Scree/Singular Value Plot
As with the toy example, assume that the 94,178-dimensional space is excessive. You would like to reduce the dimension of the image space. First compute its intrinsic dimensionality. For this application, "good enough" means capturing 95% of the variance in the dataset. How many dimensions are needed to capture at least 95% of the variance in the provided dataset? Store your answer in numDimsNeeded. (Hint: A Scree plot may be helpful, though visual inspection of a graph may not be precise enough.)

In [ ]:
n_components = 170
pca = PCA(n_components=n_components)
# pca = PCA()
pca.fit(images)

cumulative_explained_var_ratio = np.cumsum(pca.explained_variance_ratio_)
number_of_vars_to_95 = np.argmax(cumulative_explained_var_ratio>.95)
print(pca.n_samples_)
print(pca.n_features_in_)
# print(cumulative_explained_var_ratio)
# print(number_of_vars_to_95)

plt.figure(figsize=(4,4))
plt.pie(pca.explained_variance_ratio_)
plt.title("Principal Component relative values")
plt.show()


plt.figure()
plt.plot(cumulative_explained_var_ratio,label="Cumulative Explained Variance")
plt.ylabel("Explained Variance")
plt.xlabel("Principal Components")
# plt.xticks(np.arange(n_components))
plt.vlines(number_of_vars_to_95,
        np.min(cumulative_explained_var_ratio),1.0,
        colors="k",linestyles='dashed',
        label = f"95% Explained Variance @ {number_of_vars_to_95}")
plt.legend()
plt.title("PCA Cumulative Explained Variance")
plt.show()

**Question:** Approximately how many components dominate the space? What does this tell us about the intrinsic dimensionality of the space?

**Response**:

168 components explain 95% of the variance. Intrinsic dimensionality is therefore 168 (depending on where you set your threshhold)

### Task 2: Principal Components
Now plot the first 5 principal components. Hint: looking at each of these top 5 principal components; do they make sense physically, in terms of what it means for where material in the bridge is placed? Compare, for example, the differences between the 1st and 2nd principal component?

In [ ]:
# comp_imgs = components_.reshape((components_.shape[0], 217, 434))

for n_components in range(2,5):
    estimator = PCA(n_components=n_components, whiten=False)
    estimator.fit(images)  # fit PCA with fewer components to illustrate reconstructions
    # pick an index to reconstruct (e.g., first image)
    comp_imgs = estimator.components_.reshape((estimator.components_.shape[0], 217, 434))
    idx = 0
    x = images[idx]  # original flattened image (shape: n_features)
    # project to low-dim: z = (x - mean) @ components_.T
    z = (x - estimator.mean_) @ estimator.components_.T  # shape (n_components,)
    # reconstruct from the low-dim code: x_hat = z @ components + mean
    x_hat = z @ estimator.components_ + estimator.mean_  # shape (n_features,)

    # reshape for visualization
    img_orig = x.reshape(217, 434)
    img_recon = x_hat.reshape(217, 434)
    img_mean = estimator.mean_.reshape(217, 434)

    # Now plot original, mean, and reconstructed images side-by-side
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1)
    plt.imshow(img_orig, cmap='gray')
    plt.title('Original')
    plt.axis('off')

    plt.subplot(1,3,2)
    plt.imshow(img_mean, cmap='gray')
    plt.title('Mean')
    plt.axis('off')

    plt.subplot(1,3,3)
    plt.imshow(img_recon, cmap='gray')
    plt.title(f'Reconstructed (k={n_components})')
    plt.axis('off')
    plt.suptitle(f'PCA Reconstruction using {n_components} components')
    plt.show()

# visualize first 5 principal components as images (signed weights)
num_show = 5
plt.figure(figsize=(15,3))
for i in range(num_show):
    plt.subplot(1, num_show, i+1)
    plt.imshow(pca.components_[i].reshape(217,434), cmap='RdBu_r')
    plt.title(f'PC {i+1}')
    plt.axis('off')
plt.suptitle('First 5 PCA components (as images)', y=1.02)
plt.show()